In [1]:
#%%

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',500)

from html.parser import HTMLParser
html_parser = HTMLParser()

# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
#pd.set_option('max_colwidth',100)

In [4]:
#读取数据
df_o_c_1 = pd.read_csv('./data/covid19_comment_1.csv', encoding='utf-8')
df_o_c_2 = pd.read_csv('./data/covid19_comment_2.csv', encoding='utf-8')
df_o_c_3 = pd.read_csv('./data/covid19_comment_3.csv', encoding='utf-8')
df_o_c_4 = pd.read_csv('./data/covid19_comment_4.csv', encoding='utf-8')
df_o_c_5 = pd.read_csv('./data/covid19_comment_5.csv', encoding='utf-8')

df_o_s_1 = pd.read_csv('./data/covid19_submission_1.csv', encoding='utf-8')
df_o_s_2 = pd.read_csv('./data/covid19_submission_2.csv', encoding='utf-8')
df_o_s_3 = pd.read_csv('./data/covid19_submission_3.csv', encoding='utf-8')
df_o_s_4 = pd.read_csv('./data/covid19_submission_4.csv', encoding='utf-8')
df_o_s_5 = pd.read_csv('./data/covid19_submission_5.csv', encoding='utf-8')

In [5]:
list_o_s_1=df_o_s_1.title.values.tolist()
list_o_c_1=df_o_c_1.body.values.tolist()
print(len(list_o_s_1),len(list_o_c_1))
list_sid_1=df_o_s_1.id.values.tolist()
list_cid_1=df_o_c_1.id.values.tolist()

2113 23359


In [6]:
list_o_s_2=df_o_s_2.title.values.tolist()
list_o_c_2=df_o_c_2.body.values.tolist()
print(len(list_o_s_2),len(list_o_c_2))
list_sid_2=df_o_s_2.id.values.tolist()
list_cid_2=df_o_c_2.id.values.tolist()

2372 25041


In [7]:
list_o_s_3=df_o_s_3.title.values.tolist()
list_o_c_3=df_o_c_3.body.values.tolist()
print(len(list_o_s_3),len(list_o_c_3))
list_sid_3=df_o_s_3.id.values.tolist()
list_cid_3=df_o_c_3.id.values.tolist()

1878 17771


In [8]:
list_o_s_4=df_o_s_4.title.values.tolist()
list_o_c_4=df_o_c_4.body.values.tolist()
print(len(list_o_s_4),len(list_o_c_4))
list_sid_4=df_o_s_4.id.values.tolist()
list_cid_4=df_o_c_4.id.values.tolist()

1712 13778


In [9]:
list_o_s_5=df_o_s_5.title.values.tolist()
list_o_c_5=df_o_c_5.body.values.tolist()
print(len(list_o_s_5),len(list_o_c_5))
list_sid_5=df_o_s_5.id.values.tolist()
list_cid_5=df_o_c_5.id.values.tolist()


783 6223


In [10]:
#清理html标记
import html
def clearhtml(data):
    lis=[]
    for sent in data:
        s=html.unescape(str(sent))
        lis.append(s)
    return lis

In [11]:
list_html_s_1=clearhtml(list_o_s_1)
list_html_s_2=clearhtml(list_o_s_2)
list_html_s_3=clearhtml(list_o_s_3)
list_html_s_4=clearhtml(list_o_s_4)
list_html_s_5=clearhtml(list_o_s_5)

list_html_c_1=clearhtml(list_o_c_1)
list_html_c_2=clearhtml(list_o_c_2)
list_html_c_3=clearhtml(list_o_c_3)
list_html_c_4=clearhtml(list_o_c_4)
list_html_c_5=clearhtml(list_o_c_5)

In [12]:
#清理bot等
def find(data):
    lis=[]
    i=0
    for sent in data:
        if(len(re.findall('I am a bot,',str(sent)))!=0) or (len(re.findall(r'Your comment has been removed',str(sent)))!=0) or (len(re.findall(r'If you believe we made a mistake, please message the moderators',str(sent)))!=0) or (len(re.findall(r'This news source is unreliable',str(sent)))!=0) or (len(re.findall('your reminder arrives',str(sent)))!=0) or (len(re.findall('Heres your reminder',str(sent)))!=0) or (len(re.findall('delay to ingest comments',str(sent)))!=0) or (len(re.findall('You can also use this tool to immediately ingest reminder from Reddit link.',str(sent)))!=0) or (len(re.findall('you can use our tool to immediately create reminder from comment link',str(sent)))!=0) or (len(re.findall('In fact, heres the checklist',str(sent)))!=0) or (len(re.findall('You should contribute only high quality information.',str(sent)))!=0) or (len(re.findall('Here we allow the following types of posts reputable news sources this includes',str(sent)))!=0) or (len(re.findall('Reddit has a',str(sent)))!=0) or (len(re.findall('Reminddit Create Reminder Your Reminders Questions',str(sent)))!=0) or (len(re.findall('Aceelerated Mobile Pages AMP is a Google',str(sent)))!=0) or (len(re.findall('PSA anytime you find an article behind a paywall',str(sent)))!=0) or (len(re.findall('Thank you for your cooperation and for making r Coronavirus reliable',str(sent)))!=0) or (len(re.findall('Please resubmit your post',str(sent)))!=0) or (len(re.findall('Please avoid off topic political',str(sent)))!=0) or (len(re.findall('This information has already been posted',str(sent)))!=0) or (len(re.findall('Your post has been removed',str(sent)))!=0) or (len(re.findall('Your comment or post has been removed',str(sent)))!=0) or (len(re.findall('The title of your post has been edited',str(sent)))!=0) or (len(re.findall('Your submission has been removed',str(sent)))!=0) or (len(re.findall('Your post or comment has been removed',str(sent)))!=0) or (len(re.findall('You should contribute only high quality information.',str(sent)))!=0) or (len(re.findall('please message the moderators',str(sent)))!=0) or (len(re.findall('delay fetching comments',str(sent)))!=0) or (len(re.findall('Please avoid off topic political discussions.',str(sent)))!=0) or (len(re.findall('Our policy on political posts',str(sent)))!=0) or (len(re.findall('Heres your reminder from',str(sent)))!=0) or (len(re.findall('Im a bot',str(sent)))!=0) or (len(re.findall('This bot ',str(sent)))!=0) or (len(re.findall('This article was originally published',str(sent)))!=0) or (len(re.findall('You can still use CommonLit',str(sent)))!=0) or (len(re.findall('If you believe we made a mistake',str(sent)))!=0) or (len(re.findall('Why is The WHO pages contain up to date and global information',str(sent)))!=0):
            lis.append(i)
        i=i+1
    return lis
find_c_1=find(list_html_c_1)
find_c_2=find(list_html_c_2)
find_c_3=find(list_html_c_3)
find_c_4=find(list_html_c_4)
find_c_5=find(list_html_c_5)

In [13]:
df_d_c_1=df_o_c_1.drop(find_c_1)
df_d_c_1=df_d_c_1.reset_index(drop=True)

df_d_c_2=df_o_c_2.drop(find_c_2)
df_d_c_2=df_d_c_2.reset_index(drop=True)

df_d_c_3=df_o_c_3.drop(find_c_3)
df_d_c_3=df_d_c_3.reset_index(drop=True)

df_d_c_4=df_o_c_4.drop(find_c_4)
df_d_c_4=df_d_c_4.reset_index(drop=True)

df_d_c_5=df_o_c_5.drop(find_c_5)
df_d_c_5=df_d_c_5.reset_index(drop=True)

In [14]:
#清理所有
#只需要dict-dataframe；两个dataframe利用id一合并，删掉原先的body，删掉空的，得到一个完美的dataframe！
dict_s_1=dict(zip(list_sid_1,list_html_s_1))
dict_c_1=dict(zip(list_cid_1,list_html_c_1))
dict_s_2=dict(zip(list_sid_2,list_html_s_2))
dict_c_2=dict(zip(list_cid_2,list_html_c_2))
dict_s_3=dict(zip(list_sid_3,list_html_s_3))
dict_c_3=dict(zip(list_cid_3,list_html_c_3))
dict_s_4=dict(zip(list_sid_4,list_html_s_4))
dict_c_4=dict(zip(list_cid_4,list_html_c_4))
dict_s_5=dict(zip(list_sid_5,list_html_s_5))
dict_c_5=dict(zip(list_cid_5,list_html_c_5))

In [15]:
def clear(data):
#匹配网址 https://blog.csdn.net/qq569699973/article/details/94636893?utm_medium=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromMachineLearnPai2-1.control&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromMachineLearnPai2-1.control
    data1=re.sub('([hH][tT]{2}[pP]://|[hH][tT]{2}[pP][sS]://|[wW]{3}.|[wW][aA][pP].|[fF][tT][pP].|[fF][iI][lL][eE].)[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]','',str(data))
    data2=re.sub('([a-z0-9--]{1,200})\.([a-z]{2,10})(\.[a-z]{2,10})?', '',str(data1))
    
    
    #去除句子中连续重复单词，保留两个
    data3=re.sub(r'\b(\w+\s*)\1{1,}', '\\1',str(data2))
    #把botttttttttt变成bot
    data4=re.sub('([0-9a-zA-Z])\\1{2}','',str(data3))

    data5=re.sub(r'[^a-zA-Z0-9,.\'!?]+',' ',str(data4))

    # Remove Emails
    data6=re.sub('\S*@\S*\s?', '', str(data5))

    # Remove new line characters
    data7=re.sub('\s+', ' ',str(data6))

    # Remove distracting single quotes
    data8=re.sub("\'", "", str(data7))

    #data = [re.sub('removed', ' ',str(sent)) for sent in data]
    #去空
#    result = [x.strip() for x in data if x.strip()!='']
    
    return data8

In [16]:
for i in dict_s_1:
    dict_s_1[i]=clear(dict_s_1[i])
for j in dict_c_1:
    dict_c_1[j]=clear(dict_c_1[j])

<ipython-input-15-b5b4165674c3>:4: FutureWarning: Possible set difference at position 8
  data2=re.sub('([a-z0-9--]{1,200})\.([a-z]{2,10})(\.[a-z]{2,10})?', '',str(data1))


In [17]:
print(len(dict_s_1),len(dict_c_1))

2113 23359


In [18]:
for i in dict_s_2:
    dict_s_2[i]=clear(dict_s_2[i])
for j in dict_c_2:
    dict_c_2[j]=clear(dict_c_2[j])
print(len(dict_s_2),len(dict_c_2))

2372 25041


In [19]:
for i in dict_s_3:
    dict_s_3[i]=clear(dict_s_3[i])
for j in dict_c_3:
    dict_c_3[j]=clear(dict_c_3[j])
print(len(dict_s_3),len(dict_c_3))

1878 17771


In [20]:
for i in dict_s_4:
    dict_s_4[i]=clear(dict_s_4[i])
for j in dict_c_4:
    dict_c_4[j]=clear(dict_c_4[j])
print(len(dict_s_4),len(dict_c_4))

1712 13778


In [21]:
for i in dict_s_5:
    dict_s_5[i]=clear(dict_s_5[i])
for j in dict_c_5:
    dict_c_5[j]=clear(dict_c_5[j])
print(len(dict_s_5),len(dict_c_5))

783 6223


In [22]:
df_p_c_1=pd.DataFrame(pd.Series(dict_c_1),columns=['doc'])
df_p_c_1=df_p_c_1.reset_index().rename(columns={'index':'id'})

In [23]:
df_p_c_1.head(10)

,id,doc
0,fsfki8r,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies."
1,fsfklr0,Please read the comment you just replied to.
2,fsfl5mc,"We conclude from this observational study that 1. The prevalence of COVID 19 on affected cruise ships is likely to be significantly underestimated, and strategies are needed to assess and monitor all passengers to prevent community transmission after disembarkation. 1 2. Rapid Ab COVID 19 testing of patients in the acute phase is unreliable. 2 3. The majority of COVID 19 positive patients were asymptomatic 81 . 4. The presence of discordant COVID 19 results in numerous cabins suggests that ..."
3,fsfl92e,"I dont think there were sufficient tests available, and the tests that were available werent all that accurate so that the false positives would have absolutely overwhelmed the true positives, which would have told us nothing. When you have a test with a high false positive reading, about the only thing its good for is confirming a case you already have strong evidence for independent of the test. That meant travel to China."
4,fsflan7,I doubt it. The measures the govt implemented were pretty much accurate with the deaths they were reporting.
5,fsfld6t,"At least its outside, that should make it a little better."
6,fsflhz5,"Updated the title to reflect mod preference reposting the translated version below. Also, it doesnt come across well in the translation, but this is from a review of 2500 X rays from November 2019 30 April 20. Press release Coronavirus COVID19 07 May 20 Albert Schweitzer Hospital In the medical imaging department of the Albert Schweitzer Hospital in Colmar, Doctor Michel Schmitt, chief physician of the medical imaging department, is currently analyzing the whole scanner photos taken in the e..."
7,fsflijr,"It appears that your article has already been submitted Rule 5 . Exceptions will be considered for posts that add significant context or new content that was absent from the previous post. If you believe we made a mistake, please message the moderators . Thanks for keeping r COVID19 informative!"
8,fsflxtb,"In rare cases like this where the report doesnt have an obvious title to it, mods will give an acceptable title for posting so no need to report this for having an edited title Also remember this is a science sub, so cite your sources and no political or anecdotal discussions."
9,fsfmhoy,I have. Your comment s off topic for discussion here. You need to stick to discussing the paper itself not offer other ideas for improving health... whether theyve been proven to be effective or not. Its not relevant to the original paper.


In [24]:
df_p_s_1=pd.DataFrame(pd.Series(dict_s_1),columns=['doc'])
df_p_s_1=df_p_s_1.reset_index().rename(columns={'index':'id'})

In [25]:
df_p_s_1.head(10)

,id,doc
0,gu1ouy,COVID 19 in the footsteps of Ernest Shackleton
1,gu1pvy,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20"
2,gu1y27,Commentary on Hydroxychloroquine and azithromycin as a treatment of COVID 19 results of an open label non randomized clinical trial by Gautret et al
3,gu32ky,The CDC has recently made it much more difficult to directly compare influenza pneumonia deaths from past years. Charts directly comparing week by week death rates are suddenly nowhere to be found but a little digging shows pneumonia flu deaths are 4x previous years. See for yourself
4,gu50mx,QMRA of SARS CoV 2 for workers in wastewater treatment plants
5,gu59o9,212 06
6,gu5a2c,Azithromycin and Hydroxychloroquine Accelerate Recovery of Outpatients with Mild Moderate COVID 19
7,gu5tmi,Laboratory abnormalities in children with mild and severe coronavirus disease 2019 COVID 19 A pooled analysis and review
8,gu5x5g,Analysis of clinical and demographic heterogeneity of patients dying from COVID 19 in Brazil versus China and Italy
9,gu5ycs,Covid19 Virtual Testing


In [27]:
df_p_c_2=pd.DataFrame(pd.Series(dict_c_2),columns=['doc'])
df_p_c_2=df_p_c_2.reset_index().rename(columns={'index':'id'})
df_p_s_2=pd.DataFrame(pd.Series(dict_s_2),columns=['doc'])
df_p_s_2=df_p_s_2.reset_index().rename(columns={'index':'id'})

In [28]:
df_p_c_3=pd.DataFrame(pd.Series(dict_c_3),columns=['doc'])
df_p_c_3=df_p_c_3.reset_index().rename(columns={'index':'id'})
df_p_s_3=pd.DataFrame(pd.Series(dict_s_3),columns=['doc'])
df_p_s_3=df_p_s_3.reset_index().rename(columns={'index':'id'})

In [29]:
df_p_c_4=pd.DataFrame(pd.Series(dict_c_4),columns=['doc'])
df_p_c_4=df_p_c_4.reset_index().rename(columns={'index':'id'})
df_p_s_4=pd.DataFrame(pd.Series(dict_s_4),columns=['doc'])
df_p_s_4=df_p_s_4.reset_index().rename(columns={'index':'id'})

In [26]:
df_p_c_5=pd.DataFrame(pd.Series(dict_c_5),columns=['doc'])
df_p_c_5=df_p_c_5.reset_index().rename(columns={'index':'id'})
df_p_s_5=pd.DataFrame(pd.Series(dict_s_5),columns=['doc'])
df_p_s_5=df_p_s_5.reset_index().rename(columns={'index':'id'})

In [31]:
df_c_1=pd.merge(df_p_c_1,df_d_c_1)
len(df_c_1)

21873

In [32]:
df_s_1=pd.merge(df_p_s_1,df_o_s_1)
len(df_s_1)

2113

In [34]:
df_c_2=pd.merge(df_p_c_2,df_d_c_2)
df_s_2=pd.merge(df_p_s_2,df_o_s_2)
print(len(df_s_2),len(df_c_2))

2372 23611


In [35]:
df_c_3=pd.merge(df_p_c_3,df_d_c_3)
df_s_3=pd.merge(df_p_s_3,df_o_s_3)

In [36]:
df_c_4=pd.merge(df_p_c_4,df_d_c_4)
df_s_4=pd.merge(df_p_s_4,df_o_s_4)

In [37]:
df_c_5=pd.merge(df_p_c_5,df_d_c_5)
df_s_5=pd.merge(df_p_s_5,df_o_s_5)

In [39]:
df_s=pd.concat([df_s_1,df_s_2,df_s_3,df_s_4,df_s_5])
df_c=pd.concat([df_c_1,df_c_2,df_c_3,df_c_4,df_c_5])
df_s=df_s.reset_index(drop=True)
df_c=df_c.reset_index(drop=True)

In [40]:
print(len(df_s),len(df_c))

8858 80751


In [66]:
df_c.head(10)

,id,doc,subreddit,subreddit_id,link_id,parent_id,body,score,author_fullname,created,permalink,total_awards_received
0,fsfki8r,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies.",COVID19,t5_2f4l19,t3_gtypct,t1_fsfjnsj,"Is there any statistical evidence about the link to smoking (and/or vaping)?\n\nMy entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet I've only seen anecdotal stories and a few very small studies.",1,t2_157wj,2020-06-01 00:02:28,/r/COVID19/comments/gtypct/an_inflammatory_cytokine_signature_helps_predict/fsfki8r/,0
1,fsfklr0,Please read the comment you just replied to.,COVID19,t5_2f4l19,t3_gtfmgg,t1_fsf2g8n,Please reread the comment you just replied to.,1,t2_13v58r,2020-06-01 00:03:22,/r/COVID19/comments/gtfmgg/predictors_for_severe_covid19_infection/fsfklr0/,0
2,fsfl5mc,"We conclude from this observational study that 1. The prevalence of COVID 19 on affected cruise ships is likely to be significantly underestimated, and strategies are needed to assess and monitor all passengers to prevent community transmission after disembarkation. 1 2. Rapid Ab COVID 19 testing of patients in the acute phase is unreliable. 2 3. The majority of COVID 19 positive patients were asymptomatic 81 . 4. The presence of discordant COVID 19 results in numerous cabins suggests that ...",COVID19,t5_2f4l19,t3_gu1ouy,t3_gu1ouy,"\n\nWe conclude from this observational study that\n\n1. The prevalence of COVID-19 on affected cruise ships is likely to be significantly underestimated, and strategies are needed to assess and monitor all passengers to prevent community transmission after disembarkation.[1](https://thorax.bmj.com/content/early/2020/05/27/thoraxjnl-2020-215091#ref-1)\n2. Rapid Ab COVID-19 testing of patients in the acute phase is unreliable.[2](https://thorax.bmj.com/content/early/2020/05/27/thoraxjnl-2020...",1,t2_2sl8h7ex,2020-06-01 00:08:26,/r/COVID19/comments/gu1ouy/covid19_in_the_footsteps_of_ernest_shackleton/fsfl5mc/,0
3,fsfl92e,"I dont think there were sufficient tests available, and the tests that were available werent all that accurate so that the false positives would have absolutely overwhelmed the true positives, which would have told us nothing. When you have a test with a high false positive reading, about the only thing its good for is confirming a case you already have strong evidence for independent of the test. That meant travel to China.",COVID19,t5_2f4l19,t3_gtnk6w,t1_fse0k3y,"I don't think there were sufficient tests available, and the tests that were available weren't all that accurate so that the false positives would have absolutely overwhelmed the true positives, which would have told us nothing.\n\nWhen you have a test with a high false positive reading, about the only thing it's good for is confirming a case you already have strong evidence for independent of the test. That meant travel to China.",1,t2_bpwrp,2020-06-01 00:09:19,/r/COVID19/comments/gtnk6w/evidence_for_limited_early_spread_of_covid19/fsfl92e/,0
4,fsflan7,I doubt it. The measures the govt implemented were pretty much accurate with the deaths they were reporting.,COVID19,t5_2f4l19,t3_gqbdmq,t1_fsdgazn,I doubt it. The measures the gov't implemented were pretty much accurate with the deaths they were reporting.,1,t2_dhkvi9s,2020-06-01 00:09:43,/r/COVID19/comments/gqbdmq/weekly_question_thread_week_of_may_25/fsflan7/,0
5,fsfld6t,"At least its outside, that should make it a little better.",COVID19,t5_2f4l19,t3_gqbdmq,t1_fse6qsp,"At least its outside, that should make it a little better.",1,t2_2sl8h7ex,2020-06-01 00:10:20,/r/COVID19/comments/gqbdmq/weekly_question_thread_week_of_may_25/fsfld6t/,0
6,fsflhz5,"Updated the title to 

In [67]:
doc_s=list(df_s['doc'])
df_doc_s=pd.DataFrame(doc_s, columns=['doc']) 
df_doc_s.head(10)

,doc
0,COVID 19 in the footsteps of Ernest Shackleton
1,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20"
2,Commentary on Hydroxychloroquine and azithromycin as a treatment of COVID 19 results of an open label non randomized clinical trial by Gautret et al
3,The CDC has recently made it much more difficult to directly compare influenza pneumonia deaths from past years. Charts directly comparing week by week death rates are suddenly nowhere to be found but a little digging shows pneumonia flu deaths are 4x previous years. See for yourself
4,QMRA of SARS CoV 2 for workers in wastewater treatment plants
5,212 06
6,Azithromycin and Hydroxychloroquine Accelerate Recovery of Outpatients with Mild Moderate COVID 19
7,Laboratory abnormalities in children with mild and severe coronavirus disease 2019 COVID 19 A pooled analysis and review
8,Analysis of clinical and demographic heterogeneity of patients dying from COVID 19 in Brazil versus China and Italy
9,Covid19 Virtual Testing


In [68]:
doc_c=list(df_c['doc'])
df_doc_c=pd.DataFrame(doc_c, columns=['doc']) 
df_doc_c.head(10)

,doc
0,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies."
1,Please read the comment you just replied to.
2,"We conclude from this observational study that 1. The prevalence of COVID 19 on affected cruise ships is likely to be significantly underestimated, and strategies are needed to assess and monitor all passengers to prevent community transmission after disembarkation. 1 2. Rapid Ab COVID 19 testing of patients in the acute phase is unreliable. 2 3. The majority of COVID 19 positive patients were asymptomatic 81 . 4. The presence of discordant COVID 19 results in numerous cabins suggests that ..."
3,"I dont think there were sufficient tests available, and the tests that were available werent all that accurate so that the false positives would have absolutely overwhelmed the true positives, which would have told us nothing. When you have a test with a high false positive reading, about the only thing its good for is confirming a case you already have strong evidence for independent of the test. That meant travel to China."
4,I doubt it. The measures the govt implemented were pretty much accurate with the deaths they were reporting.
5,"At least its outside, that should make it a little better."
6,"Updated the title to reflect mod preference reposting the translated version below. Also, it doesnt come across well in the translation, but this is from a review of 2500 X rays from November 2019 30 April 20. Press release Coronavirus COVID19 07 May 20 Albert Schweitzer Hospital In the medical imaging department of the Albert Schweitzer Hospital in Colmar, Doctor Michel Schmitt, chief physician of the medical imaging department, is currently analyzing the whole scanner photos taken in the e..."
7,"In rare cases like this where the report doesnt have an obvious title to it, mods will give an acceptable title for posting so no need to report this for having an edited title Also remember this is a science sub, so cite your sources and no political or anecdotal discussions."
8,I have. Your comment s off topic for discussion here. You need to stick to discussing the paper itself not offer other ideas for improving health... whether theyve been proven to be effective or not. Its not relevant to the original paper.
9,Lots of big pharma employees and investors in this sub who upvote or downvote with their wallets.


In [46]:
#清理author_fullname
def findwrong(data):
    lis=[]
    i=0
    for sent in data:
        if(len(re.findall('Your submission has been removed',str(sent)))!=0) or (len(re.findall('your submission has been removed',str(sent)))!=0):
            lis.append(i)
        i=i+1
    return lis

In [48]:
nice_=findwrong(doc)
len(nice_)

0

In [69]:
#利用LIWC再清理post
df_doc_c.to_csv('./doc_c.csv')
df_doc_s.to_csv('./doc_s.csv')
print(len(df_doc_c),len(df_doc_s))

80751 8858


In [80]:
df_LIWC_s=pd.read_csv('./liwc_results_doc_s.csv', encoding='utf-8')
df_LIWC_s=df_LIWC_s.drop([0])
df_LIWC_s.reset_index(drop=True, inplace=True)
df_LIWC_s.head(5)

,A,B,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,0.0,COVID 19 in the footsteps of Ernest Shackleton,8,99.00,50.00,7.84,25.77,8.00,25.00,37.50,37.50,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,12.50,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,12.50,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,12.50,0.0,12.50,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20",14,99.00,50.00,1.06,25.77,14.00,28.57,64.29,42.86,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,14.29,21.43,7.14,0.00,0.00,0.00,7.14,0.00,0.00,0.0,7.14,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,7.14,0.00,0.00,0.0,7.14,0.00,0.00,7.14,0.00,0.0,7.14,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,7.14,7.14,0.0,7.14,0.00,7.14,0.00,7.14,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,7.14,0.00,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,Commentary on Hydroxychloroquine and azithromycin as a treatment of COVID 19 results of an open label non randomized clinical trial by Gautret et al,24,99.00,66.17,1.79,25.77,24.00,33.33,50.00,33.33,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,8.33,20.83,0.00,0.00,8.33,0.00,4.17,4.17,4.17,0.0,4.17,0.00,0.00,0.0,0.00,0.0,0.0,0.0,4.17,0.0,0.0,0.0,0.0,4.17,0.00,4.17,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,4.17,0.0,4.17,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,4.17,0.00,8.33,0.0,8.33,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,The CDC has recently made it much more difficult to directly compare influenza pneumonia deaths from past years. Charts directly comparing week by week death rates are suddenly nowhere to be found but a little digging shows pneumonia flu deaths are 4x previous years. See for yourself,47,88.34,50.00,92.84,4.35,15.67,29.79,85.11,34.04,4.26,2.13,0.0,0.0,2.13,0.0,0.0,2.13,4.26,10.64,8.51,2.13,2.13,2.13,14.89,10.64,8.51,0.0,0.00,4.26,2.13,0.0,2.13,0.0,0.0,0.0,2.13,0.0,0.0,0.0,0.0,12.77,2.13,2.13,0.0,0.00,6.38,2.13,4.26,4.26,0.0,0.00,2.13,0.0,2.13,0.0,0.0,2.13,0.0,0.00,0.00,0.0,2.13,8.51,10.64,0.00,25.53,0.0,8.51,17.02,0.00,2.13,0.00,0.0,0.0,6.38,0.0,0.0,0.0,0.0,0.0,0.0,4.26,4.26,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,QMRA of SARS CoV 2 for workers in wastewater treatment plants,11,99.00,50.00,2.40,25.77,11.00,27.27,36.36,27.27,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,27.27,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,9.09,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,9.09,0.0,9.09,9.09,0.0,0.00,0.00,0.00,0.00,9.09,0.0,9.09,0.00,9.09,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
df_LIWC_c=pd.read_csv('./liwc_results_doc_c.csv', encoding='utf-8')
df_LIWC_c=df_LIWC_c.drop([0])
df_LIWC_c.reset_index(drop=True, inplace=True)
df_LIWC_c.head(5)

,A,B,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,0.0,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies.",49,57.33,27.03,82.17,25.77,24.50,24.49,89.80,53.06,10.20,4.08,4.08,0.00,0.00,0.0,0.00,6.12,10.20,10.20,6.12,10.20,12.24,0.00,12.24,6.12,4.08,0.00,0.00,6.12,0.00,0.00,0.00,0.00,0.0,0.00,2.04,0.0,0.0,0.0,0.0,24.49,10.20,2.04,0.00,12.24,2.04,4.08,2.04,2.04,0.0,0.0,6.12,2.04,0.00,0.0,4.08,2.04,0.00,0.00,2.04,0.00,0.00,2.04,8.16,0.00,8.16,0.00,6.12,2.04,2.04,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,6.12,2.04,2.04,0.0,0.0,2.04,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,Please read the comment you just replied to.,8,93.26,99.00,1.00,99.00,8.00,25.00,100.00,50.00,12.50,12.50,0.00,0.00,12.50,0.0,0.00,0.00,12.50,12.50,0.00,12.50,0.00,0.00,12.50,0.00,0.00,0.00,0.00,0.00,12.50,12.50,0.00,0.00,0.0,0.00,37.50,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.50,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,12.50,12.50,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,"We conclude from this observational study that 1. The prevalence of COVID 19 on affected cruise ships is likely to be significantly underestimated, and strategies are needed to assess and monitor all passengers to prevent community transmission after disembarkation. 1 2. Rapid Ab COVID 19 testing of patients in the acute phase is unreliable. 2 3. The majority of COVID 19 positive patients were asymptomatic 81 . 4. The presence of discordant COVID 19 results in numerous cabins suggests that ...",121,94.01,66.04,4.39,25.77,11.00,29.75,64.46,38.84,4.96,0.83,0.00,0.83,0.00,0.0,0.00,4.13,4.96,14.88,9.92,1.65,1.65,0.83,15.70,4.13,1.65,0.00,10.74,2.48,1.65,0.83,0.83,0.00,0.0,0.83,4.13,0.0,0.0,0.0,0.0,9.09,1.65,1.65,0.83,3.31,1.65,0.00,0.83,0.83,0.0,0.0,0.83,0.00,0.83,0.0,0.00,4.96,1.65,0.83,0.83,0.83,0.83,3.31,7.44,1.65,9.92,2.48,4.13,3.31,4.96,0.83,0.0,0.0,0.0,0.00,0.83,0.0,0.83,0.83,0.0,0.0,9.92,9.09,0.83,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,"I dont think there were sufficient tests available, and the tests that were available werent all that accurate so that the false positives would have absolutely overwhelmed the true positives, which would have told us nothing. When you have a test with a high false positive reading, about the only thing its good for is confirming a case you already have strong evidence for independent of the test. That meant travel to China.",73,28.82,60.81,2.88,99.00,24.33,20.55,89.04,60.27,15.07,5.48,1.37,1.37,2.74,0.0,0.00,9.59,10.96,8.22,15.07,9.59,4.11,4.11,21.92,5.48,0.00,2.74,0.00,1.37,9.59,8.22,1.37,1.37,0.0,1.37,5.48,0.0,0.0,0.0,0.0,20.55,4.11,1.37,2.74,0.00,10.96,1.37,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,9.59,1.37,0.00,2.74,5.48,0.00,8.22,10.96,0.00,5.48,1.37,1.37,2.74,5.48,1.37,0.0,0.0,0.0,0.00,1.37,0.0,0.00,1.37,0.0,0.0,8.22,4.11,4.11,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,I doubt it. The measures the govt implemented were pretty much accurate with the deaths they were reporting.,18,86.43,50.00,1.00,25.77,9.00,22.22,83.33,50.00,16.67,11.11,5.56,0.00,0.00,0.0,5.56,5.56,16.67,5.56,11.11,0.00,0.00,0.00,11.11,0.00,0.00,0.00,0.00,5.56,11.1

In [82]:
df_LIWC_c.drop(['A'],axis=1,inplace=True)
df_LIWC_c.head(2)

,B,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies.",49,57.33,27.03,82.17,25.77,24.5,24.49,89.8,53.06,10.2,4.08,4.08,0.0,0.0,0.0,0.0,6.12,10.2,10.2,6.12,10.2,12.24,0.0,12.24,6.12,4.08,0.0,0.0,6.12,0.0,0.0,0.0,0.0,0.0,0.0,2.04,0.0,0.0,0.0,0.0,24.49,10.2,2.04,0.0,12.24,2.04,4.08,2.04,2.04,0.0,0.0,6.12,2.04,0.0,0.0,4.08,2.04,0.0,0.0,2.04,0.0,0.0,2.04,8.16,0.0,8.16,0.0,6.12,2.04,2.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.12,2.04,2.04,0.0,0.0,2.04,0.0,0.0,0.0,0.0,0.0,0.0
1,Please read the comment you just replied to.,8,93.26,99.00,1.00,99.00,8.0,25.00,100.0,50.00,12.5,12.50,0.00,0.0,12.5,0.0,0.0,0.00,12.5,12.5,0.00,12.5,0.00,0.0,12.50,0.00,0.00,0.0,0.0,0.00,12.5,12.5,0.0,0.0,0.0,0.0,37.50,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.50,12.50,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
df_LIWC_s.drop(['A'],axis=1,inplace=True)
df_LIWC_s.head(2)

,B,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,COVID 19 in the footsteps of Ernest Shackleton,8,99.0,50.0,7.84,25.77,8.0,25.00,37.50,37.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.50,25.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,12.50,0.0,12.50,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20",14,99.0,50.0,1.06,25.77,14.0,28.57,64.29,42.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.29,21.43,7.14,0.0,0.0,0.0,7.14,0.0,0.0,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.14,0.0,0.0,0.0,7.14,0.0,0.0,7.14,0.0,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.14,7.14,0.0,7.14,0.0,7.14,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.14,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
df_LIWC_c=df_LIWC_c.reset_index(drop=True)
df_LIWC_c.head(2)

,B,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies.",49,57.33,27.03,82.17,25.77,24.5,24.49,89.8,53.06,10.2,4.08,4.08,0.0,0.0,0.0,0.0,6.12,10.2,10.2,6.12,10.2,12.24,0.0,12.24,6.12,4.08,0.0,0.0,6.12,0.0,0.0,0.0,0.0,0.0,0.0,2.04,0.0,0.0,0.0,0.0,24.49,10.2,2.04,0.0,12.24,2.04,4.08,2.04,2.04,0.0,0.0,6.12,2.04,0.0,0.0,4.08,2.04,0.0,0.0,2.04,0.0,0.0,2.04,8.16,0.0,8.16,0.0,6.12,2.04,2.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.12,2.04,2.04,0.0,0.0,2.04,0.0,0.0,0.0,0.0,0.0,0.0
1,Please read the comment you just replied to.,8,93.26,99.00,1.00,99.00,8.0,25.00,100.0,50.00,12.5,12.50,0.00,0.0,12.5,0.0,0.0,0.00,12.5,12.5,0.00,12.5,0.00,0.0,12.50,0.00,0.00,0.0,0.0,0.00,12.5,12.5,0.0,0.0,0.0,0.0,37.50,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.50,12.50,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
df_LIWC_s_processing=df_LIWC_s.drop(df_LIWC_s[df_LIWC_s.Dic < 50].index)
print(df_LIWC_s_processing.shape)
df_LIWC_c_processing=df_LIWC_c.drop(df_LIWC_c[df_LIWC_c.Dic < 50].index)
print(df_LIWC_c_processing.shape)

(6149, 94)
(79441, 94)


In [87]:
df_LIWC_s_processing=df_LIWC_s_processing.reset_index(drop=True)
df_LIWC_s_processing[6000:]

,B,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
6000,Development of a hyperimmune equine serum therapy for COVID 19 in Argentina,12,99.00,50.00,1.79,25.77,12.00,33.33,50.00,33.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.33,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.33,0.00,8.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.33,0.00,8.33,0.00,8.33,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
6001,Comic and Teacher talks about being a teacher during COVID,10,93.26,84.14,3.37,25.77,10.00,20.00,90.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,20.00,10.00,10.00,10.00,0.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,0.00,0.00,20.00,0.00,0.00,0.00,10.00,0.00,10.00,0.00,0.00,10.00,20.00,10.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
6002,"COVID 19, Type 2 Diabetes, and Associated Health Outcomes in China Results from a Nationwide Survey of 10,545 Adults",20,99.00,69.14,1.00,25.77,20.00,25.00,55.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,15.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,20.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.0,0.00,10.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,5.00,0.00,5.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,15.00,0.00,15.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
6003,A friend worked really hard on this! Show him some love COVID19 Timeline,13,82.82,93.79,17.46,99.00,6.50,15.38,92.31,38.46,15.38,7.69,0.00,0.00,0.00,7.69,0.00,7.69,7.69,7.69,0.00,7.69,0.00,0.00,7.69,7.69,0.00,0.00,0.00,7.69,7.69,7.69,0.00,0.00,0.00,0.00,23.08,0.00,7.69,0.0,7.69,15.38,0.00,0.00,0.00,7.69,0.00,7.69,15.38,7.69,0.00,7.69,7.69,0.00,0.00,0.00,0.00,23.08,15.38,7.69,0.00,0.00,0.00,7.69,0.00,0.00,15.38,0.00,7.69,7.69,7.69,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,7.69,0.00,0.00,0.0,0.0,0.00,7.69,0.0,0.0,0.0,0.0,0.0
6004,"US Patent Boss Says No Evidence Of Patents Holding Back COVID Treatments, Days Before Pharma Firms Prove Hes Wrong",19,64.39,94.27,50.35,1.00,19.00,21.05,63.16,31.58,10.53,10.53,0.00,5.26,0.00,5.26,0.00,0.00,0.00,10.53,5.26,5.26,0.00,5.26,10.53,10.53,5.26,0.00,0.00,0.00,5.26,0.00,5.26,0.00,0.00,0.00,15.79,0.00,0.00,0.0,5.26,10.53,10.53,0.00,0.00,0.00,5.26,0.00,5.26,0.00,5.26,0.00,0.00,0.00,0.00,0.00,0.00,15.79,5.26,0.00,5.26,0.00,5.26,0.00,10.53,0.00,15.79,0.00,5.26,15.79,5.26,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,5.26,0.00,5.26,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
6005,Sars Cov2 A plausible theory relating to Serotonin,8,99.00,50.00,7.84,25.77,8.00,37.50,50.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12.50,12.50,0.00,0.00,0.00,0.00,12.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,25.00,12.50,0.00,0.00,12.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
6006,Try something other than lockdown,5,52.71,1.00,99.00,25.77

In [88]:
df_LIWC_c_processing=df_LIWC_c_processing.reset_index(drop=True)
df_LIWC_c_processing[79000:]

,B,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
79000,This is wrong it absolutely can be common and likely is. Some people may be asympyomatic the first or second time. There are anecdotal reports among our hospitals of people assuming to be infected again but due to inadequate testing we are lucky to catch the few we have. There is no common thread among them other than they were lucky to be tested the first time. Have colleagues that are extremely healthy that are catching it likely for the second time but due to inadequate testing the first ...,137,21.25,61.50,65.29,25.77,17.12,18.25,94.16,61.31,16.06,7.30,1.46,4.38,0.00,0.00,1.46,8.76,4.38,10.22,18.25,4.38,5.84,2.19,23.36,6.57,0.73,0.00,3.65,2.19,5.84,2.92,2.92,2.19,0.00,1.46,8.76,0.00,0.73,0.00,0.00,18.98,2.19,0.00,1.46,8.03,2.92,6.57,1.46,1.46,0.00,0.00,2.19,0.00,2.19,0.00,0.00,12.41,5.11,3.65,0.00,2.19,1.46,2.19,16.79,2.19,14.60,0.73,3.65,8.76,3.65,0.00,0.00,0.00,0.00,0.00,0.73,0.00,0.00,0.73,0.00,0.00,5.84,5.84,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
79001,ADE that doesnt occur after antibodies fade?,7,29.30,7.67,89.63,25.77,7.00,14.29,57.14,42.86,14.29,0.00,0.00,0.00,0.00,0.00,0.00,14.29,0.00,14.29,14.29,0.00,0.00,14.29,14.29,14.29,14.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.29,0.00,28.57,0.00,0.00,28.57,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.29,0.00,0.00,0.0,0.0,14.29,0.00,0.0,0.0,0.0,0.0,0.0
79002,The D and G variants are neutralised by the same antibodies so it should have no effect.,17,27.29,27.83,1.00,25.77,17.00,17.65,70.59,58.82,5.88,0.00,0.00,0.00,0.00,0.00,0.00,5.88,11.76,5.88,17.65,5.88,11.76,5.88,17.65,5.88,5.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.76,5.88,5.88,5.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.88,5.88,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
79003,So I read these and I am still ready to dismiss those same concerns. Im happy to revisit this in 6 months to a year and recap.,27,26.77,13.32,79.42,99.00,13.50,11.11,85.19,59.26,22.22,11.11,11.11,0.00,0.00,0.00,0.00,11.11,3.70,14.81,7.41,7.41,11.11,0.00,11.11,7.41,3.70,0.00,3.70,0.00,7.41,7.41,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.41,0.00,14.81,0.00,3.70,11.11,7.41,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.41,7.41,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
79004,It s not effective.,4,1.92,1.00,99.00,25.77,4.00,25.00,75.00,50.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,25.00,0.00,0.00,0.00,0.00,0.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,25.00,25.00,0.00,0.00,0.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25.00,25.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0
79005,"That is the theory, but in practice, reinfections seem to be quite common, so the theory does not match real life.",21,55.40,7.67,68.01,25.77,21.00,9.52,85.71,52.38,4.76,0.00,0.00,0.00,0.00,0.00,0.00,4.76,9

In [89]:
#合并LIWC( df_LIWC_s_processing 和 df_LIWC_c_processing)和日期啥的（df_s 和df_c)
print(df_LIWC_s_processing.shape,df_s.shape)
print(df_LIWC_c_processing.shape,df_c.shape)

(6149, 94) (8858, 14)
(79441, 94) (80751, 12)


In [90]:
df_LIWC_s_processing.columns

Index(['B', 'WC', 'Analytic', 'Clout', 'Authentic', 'Tone', 'WPS', 'Sixltr',
       'Dic', 'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe',
       'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj',
       'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant',
       'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family',
       'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep',
       'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio',
       'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation',
       'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent',
       'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure',
       'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak',
       'assent', 'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'Colon',
       'SemiC', 'QMark', 'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth',
       'OtherP'],
  

In [93]:
LIWC_columns=['doc', 'WC', 'Analytic', 'Clout', 'Authentic', 'Tone', 'WPS', 'Sixltr',
       'Dic', 'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe',
       'they', 'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj',
       'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant',
       'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family',
       'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 'discrep',
       'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio',
       'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation',
       'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent',
       'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure',
       'home', 'money', 'relig', 'death', 'informal', 'swear', 'netspeak',
       'assent', 'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'Colon',
       'SemiC', 'QMark', 'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth',
       'OtherP']

In [94]:
df_LIWC_s_processing.columns=LIWC_columns

In [95]:
df_LIWC_s_processing.head(2)

,doc,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20",14,99.0,50.00,1.06,25.77,14.0,28.57,64.29,42.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.29,21.43,7.14,0.0,0.00,0.0,7.14,0.00,0.00,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,7.14,0.0,0.00,0.0,7.14,0.0,0.0,7.14,0.0,0.0,7.14,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,7.14,7.14,0.0,7.14,0.0,7.14,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.14,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Commentary on Hydroxychloroquine and azithromycin as a treatment of COVID 19 results of an open label non randomized clinical trial by Gautret et al,24,99.0,66.17,1.79,25.77,24.0,33.33,50.00,33.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.33,20.83,0.00,0.0,8.33,0.0,4.17,4.17,4.17,0.0,4.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17,0.0,0.0,0.0,0.0,4.17,0.0,4.17,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,4.17,0.0,4.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17,0.00,8.33,0.0,8.33,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
df_s.head(2)

,id,doc,subreddit,subreddit_id,title,author,author_fullname,num_comments,num_crossposts,permalink,score,total_awards_received,full_link,created_utc
0,gu1ouy,COVID 19 in the footsteps of Ernest Shackleton,COVID19,t5_2f4l19,COVID-19: in the footsteps of Ernest Shackleton,EthicalFrames,t2_2sl8h7ex,2,0,/r/COVID19/comments/gu1ouy/covid19_in_the_footsteps_of_ernest_shackleton/,1,0,https://www.reddit.com/r/COVID19/comments/gu1ouy/covid19_in_the_footsteps_of_ernest_shackleton/,2020-06-01 00:07:59
1,gu1pvy,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20",COVID19,t5_2f4l19,"Foundation of the House of the Diaconate of Mulhouse Press Release - May 7th, 2020",Hoosiergirl29,t2_gh14o,19,0,/r/COVID19/comments/gu1pvy/foundation_of_the_house_of_the_diaconate_of/,10,0,https://www.reddit.com/r/COVID19/comments/gu1pvy/foundation_of_the_house_of_the_diaconate_of/,2020-06-01 00:09:36


In [98]:
drop_index_s=df_LIWC_s[df_LIWC_s.Dic < 40].index
len(drop_index_s)

1376

In [99]:
drop_index_c=df_LIWC_c[df_LIWC_c.Dic < 40].index
len(drop_index_c)

889

In [100]:
df_c_processing=df_c.drop(drop_index_c)
df_c_processing=df_c_processing.reset_index(drop=True)
print(df_c_processing.shape)
print(df_LIWC_c_processing.shape)

(79862, 12)
(79441, 94)


In [101]:
df_s_processing=df_s.drop(drop_index_s)
df_s_processing=df_s_processing.reset_index(drop=True)
print(df_s_processing.shape)
print(df_LIWC_s_processing.shape)

(7482, 14)
(6149, 94)


In [102]:
df_LIWC_c_processing.columns=LIWC_columns

In [103]:
df_S_0=pd.merge(df_s_processing,df_LIWC_s_processing,left_index=True,right_index=True)

In [104]:
df_S_0.head(2)

,id,doc_x,subreddit,subreddit_id,title,author,author_fullname,num_comments,num_crossposts,permalink,score,total_awards_received,full_link,created_utc,doc_y,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,gu1pvy,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20",COVID19,t5_2f4l19,"Foundation of the House of the Diaconate of Mulhouse Press Release - May 7th, 2020",Hoosiergirl29,t2_gh14o,19,0,/r/COVID19/comments/gu1pvy/foundation_of_the_house_of_the_diaconate_of/,10,0,https://www.reddit.com/r/COVID19/comments/gu1pvy/foundation_of_the_house_of_the_diaconate_of/,2020-06-01 00:09:36,"Foundation of the House of the Diaconate of Mulhouse Press Release May 7th, 20",14,99.0,50.00,1.06,25.77,14.0,28.57,64.29,42.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.29,21.43,7.14,0.0,0.00,0.0,7.14,0.00,0.00,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,7.14,0.0,0.00,0.0,7.14,0.0,0.0,7.14,0.0,0.0,7.14,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,7.14,7.14,0.0,7.14,0.0,7.14,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.14,0.0,7.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,gu1y27,Commentary on Hydroxychloroquine and azithromycin as a treatment of COVID 19 results of an open label non randomized clinical trial by Gautret et al,COVID19,t5_2f4l19,Commentary on “Hydroxychloroquine and azithromycin as a treatment of COVID-19: results of an open label non-randomized clinical trial” by Gautret et al,RGregoryClark,t2_aek73,18,0,/r/COVID19/comments/gu1y27/commentary_on_hydroxychloroquine_and_azithromycin/,24,0,https://www.reddit.com/r/COVID19/comments/gu1y27/commentary_on_hydroxychloroquine_and_azithromycin/,2020-06-01 00:22:33,Commentary on Hydroxychloroquine and azithromycin as a treatment of COVID 19 results of an open label non randomized clinical trial by Gautret et al,24,99.0,66.17,1.79,25.77,24.0,33.33,50.00,33.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.33,20.83,0.00,0.0,8.33,0.0,4.17,4.17,4.17,0.0,4.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17,0.0,0.0,0.0,0.0,4.17,0.0,4.17,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,4.17,0.0,4.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17,0.00,8.33,0.0,8.33,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
df_C_0=pd.merge(df_c_processing,df_LIWC_c_processing,left_index=True,right_index=True)
df_C_0.head(2)

,id,doc_x,subreddit,subreddit_id,link_id,parent_id,body_x,score,author_fullname,created,permalink,total_awards_received,doc_y,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body_y,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,fsfki8r,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies.",COVID19,t5_2f4l19,t3_gtypct,t1_fsfjnsj,"Is there any statistical evidence about the link to smoking (and/or vaping)?\n\nMy entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet I've only seen anecdotal stories and a few very small studies.",1,t2_157wj,2020-06-01 00:02:28,/r/COVID19/comments/gtypct/an_inflammatory_cytokine_signature_helps_predict/fsfki8r/,0,"Is there any statistical evidence about the link to smoking and or vaping ? My entirely unscientific assumption is that a link between a virus that affects the lungs and things like smoking seems fairly likely, but as yet Ive only seen anecdotal stories and a few very small studies.",49,57.33,27.03,82.17,25.77,24.5,24.49,89.8,53.06,10.2,4.08,4.08,0.0,0.0,0.0,0.0,6.12,10.2,10.2,6.12,10.2,12.24,0.0,12.24,6.12,4.08,0.0,0.0,6.12,0.0,0.0,0.0,0.0,0.0,0.0,2.04,0.0,0.0,0.0,0.0,24.49,10.2,2.04,0.0,12.24,2.04,4.08,2.04,2.04,0.0,0.0,6.12,2.04,0.0,0.0,4.08,2.04,0.0,0.0,2.04,0.0,0.0,2.04,8.16,0.0,8.16,0.0,6.12,2.04,2.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.12,2.04,2.04,0.0,0.0,2.04,0.0,0.0,0.0,0.0,0.0,0.0
1,fsfklr0,Please read the comment you just replied to.,COVID19,t5_2f4l19,t3_gtfmgg,t1_fsf2g8n,Please reread the comment you just replied to.,1,t2_13v58r,2020-06-01 00:03:22,/r/COVID19/comments/gtfmgg/predictors_for_severe_covid19_infection/fsfklr0/,0,Please read the comment you just replied to.,8,93.26,99.00,1.00,99.00,8.0,25.00,100.0,50.00,12.5,12.50,0.00,0.0,12.5,0.0,0.0,0.00,12.5,12.5,0.00,12.5,0.00,0.0,12.50,0.00,0.00,0.0,0.0,0.00,12.5,12.5,0.0,0.0,0.0,0.0,37.50,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,12.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.50,12.50,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
print(len(df_S_0),len(df_C_0))

6149 79441


In [107]:
df_S_0.to_csv('./df_S_0.csv')
df_C_0.to_csv('./df_C_0.csv')

In [ ]:
df_LIWC_processing=df_LIWC.drop(df_LIWC[df_LIWC.Dic < 40].index)
df_LIWC_processing=df_LIWC_processing.reset_index(drop=True)
df_LIWC_processing.shape

In [ ]:
post=df_LIWC_processing.B.values.tolist()

In [ ]:
# LDA
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(post))

print(data_words[:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','removed','deleted'])

In [ ]:
#调用
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
id2word = corpora.Dictionary(data_lemmatized)

In [ ]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
import os
from gensim.models.wrappers import LdaMallet
os.environ['MALLET_HOME'] = '/home/byliu/reddit/mallet-2.0.8'
os.environ.update({'MALLET_PATH':r'/home/byliu/reddit/mallet-2.0.8'})
mallet_path =r'/home/byliu/reddit/mallet-2.0.8/bin/mallet'# update this path000000000000000000

In [ ]:
# topic=20，Coherence Score: 0.7148060963017324
prefix20='/home/byliu/reddit/prefix/202105/20'
ldamallet20=gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word, prefix=prefix20)
coherence_model_ldamallet20 = CoherenceModel(model=ldamallet20, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet20 = coherence_model_ldamallet20.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet20)

In [ ]:
frequencies1 = dict(ldamallet20.show_topics(num_topics=20,num_words = 200,formatted = False))

In [ ]:
for t in range(len(frequencies1)):
    plt.figure()
    frequencies2 = dict(frequencies1[t])
    plt.imshow(WordCloud(
                        background_color='white',
                        width=2500,
                        height=1800,
                        max_words=50,).fit_words(frequencies2))
    plt.axis("off")
    plt.title("Topic #" + str(t+1))
    plt.show()

In [ ]:
# 读取topic=20的doctopicmatrix

In [ ]:
#i=0
t=[]
with open('/home/byliu/reddit/prefix/202105/20doctopics.txt','r') as f:
    for line in f:
#        result.append(list(line.strip('\n').split(',')))
        #cleanedData = re.sub('[\n\t]', '', f.read())
#        line=re.sub('0\t0\t','',str(line))
        line=re.sub('[\n\t]',' ',str(line))
        t.append(list(line.strip().split(' ')))
#        i=i+1
#        if i>10:
#            break

In [ ]:
df_lda_doctopic_matrix=pd.DataFrame(t)

In [ ]:
df_lda_doctopic_matrix.drop([0,1],axis=1,inplace=True)
#axis=1表示取列

In [ ]:
new_col = ['topic1','topic2','topic3','topic4','topic5','topic6','topic7','topic8','topic9','topic10','topic11','topic12','topic13','topic14','topic15','topic16','topic17','topic18','topic19','topic20']
df_lda_doctopic_matrix.columns = new_col

In [ ]:
maxindex = df_lda_doctopic_matrix.astype(float).idxmax()

In [ ]:
maxindex

In [ ]:
maxindex_list=maxindex.to_list()

In [ ]:
df_LIWC_processing.iloc[maxindex_list]

In [ ]:
df_topic16=df_lda_doctopic_matrix.sort_values(by='topic16',ascending=False)

In [ ]:
df_topic16.head(10)

In [ ]:
list_t16=df_topic16[81300:81325].index

In [ ]:
df_LIWC_processing.iloc[list_t16]

In [ ]:
or (len(re.findall('Your submission has been removed',str(sent)))!=0) or (len(re.findall('Your post or comment has been removed',str(sent)))!=0) or (len(re.findall('You should contribute only high quality information.',str(sent)))!=0) or (len(re.findall('please message the moderators',str(sent)))!=0) or (len(re.findall('delay fetching comments',str(sent)))!=0) or (len(re.findall('Please avoid off topic political discussions.',str(sent)))!=0) or (len(re.findall('Our policy on political posts',str(sent)))!=0) or (len(re.findall('Heres your reminder from',str(sent)))!=0) or (len(re.findall('Im a bot',str(sent)))!=0) or (len(re.findall('This bot ',str(sent)))!=0) or (len(re.findall('This article was originally published',str(sent)))!=0) or (len(re.findall('You can still use CommonLit',str(sent)))!=0) or (len(re.findall('If you believe we made a mistake',str(sent)))!=0) or (len(re.findall('Why is The WHO pages contain up to date and global information',str(sent)))!=0)

In [ ]:
df_topic1=df_lda_doctopic_matrix.sort_values(by='topic1',ascending=False)
list_t1=df_topic1[0:25].index
df_LIWC_processing.iloc[list_t1]

In [ ]:
df_topic2=df_lda_doctopic_matrix.sort_values(by='topic2',ascending=False)
list_t2=df_topic2[0:25].index
df_LIWC_processing.iloc[list_t2]

In [ ]:
prefix19='/home/byliu/reddit/prefix/202105/19'
ldamallet19=gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=19, id2word=id2word, prefix=prefix19)
coherence_model_ldamallet19 = CoherenceModel(model=ldamallet19, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet19 = coherence_model_ldamallet19.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet19)

frequencies19 = dict(ldamallet19.show_topics(num_topics=19,num_words = 200,formatted = False))

for t in range(len(frequencies19)):
    plt.figure()
    frequencies2 = dict(frequencies19[t])
    plt.imshow(WordCloud(
                        background_color='white',
                        width=2500,
                        height=1800,
                        max_words=50,).fit_words(frequencies2))
    plt.axis("off")
    plt.title("Topic #" + str(t+1))
    plt.show()


In [ ]:
# 总网络的author量
s_author_c=df_c['author_fullname']
s_author_s=df_s['author_fullname']

In [ ]:
s_author=pd.concat([s_author_c,s_author_s])
print(len(s_author))

In [ ]:
s_author=s_author.drop_duplicates()
len(s_author)